## Optimization code for PV microgrid:

In [1]:
### Initialization:

using JuMP, Gurobi, MATLAB


In [2]:
my_start =  1
my_end   = 96

# my_start = 96
# my_end   = 96

96

In [3]:
# for my_int = my_start:my_end

my_int = my_start

while my_int <= my_end

    #----------------------------------------------------------------------------------------------------
    ### Solver Definition:
    
    m = Model(solver=GurobiSolver(Presolve=1,TimeLimit=900))
        
    #----------------------------------------------------------------------------------------------------
    ### Loading Optimization Parameters:

    mat"generate_data($my_int)"
    
    vars = read_matfile("../../Data/Generated Data/5 - Optimization/scenarios/scen_"*dec(my_int)*".mat")

    interval  = Int64(jvalue(vars["interval"]))  # Current Interval (in day, for which we are optimizing)

    N_EMS   = Int64(jvalue(vars["N_EMS"]))   # num of remaining EMS   intervals (opt. executions)
    N_intra = Int64(jvalue(vars["N_intra"])) # num of remaining Intra intervals (opt. resolution)
    N_dies  = Int64(jvalue(vars["N_dies"]))  # num of diesel generators
    N_scen  = Int64(jvalue(vars["N_scen"]))  # num of different scenations
    N_firm  = Int64(jvalue(vars["N_firm"]))  # num of FIRM Margin Constraint Intervals

    # Upper and Lower bounds:
    P_bat_min = jvalue(vars["P_bat_min"])
    P_bat_max = jvalue(vars["P_bat_max"])
    eta_bat  = jvalue(vars["eta_bat"])
    SOC_min  = jvalue(vars["SOC_min"])
    SOC_max  = jvalue(vars["SOC_max"])
    SOC_init = jvalue(vars["SOC_init"])
    Cap_bat  = jvalue(vars["Cap_bat"])
    P_dies_max = jvalue(vars["P_dies_max"])
    P_dies_min = jvalue(vars["P_dies_min"])
    P_PV_inst = jvalue(vars["P_PV_inst"])

    delta_t = jvalue(vars["delta_t"]) # 1 min

    # f_min = -100000
    f_min   = jvalue(vars["f_min"])
    
    theta_i = jvalue(vars["theta_i"])
    theta_d = jvalue(vars["theta_d"])
    theta_b = jvalue(vars["theta_b"])
    theta_p = jvalue(vars["theta_p"])
    
    U_min_def = jvalue(vars["U_min_def"]) # - my_correct*0.025
    
    psi_min_i = jvalue(vars["psi_min_i"])
    psi_min_d = jvalue(vars["psi_min_d"])
    psi_min_p = jvalue(vars["psi_min_p"])
    psi_min_b = jvalue(vars["psi_min_b"])
    psi_min_c = jvalue(vars["psi_min_c"])    
    
    psi_loss_i = jvalue(vars["psi_loss_i"])
    psi_loss_d = jvalue(vars["psi_loss_d"])
    psi_loss_p = jvalue(vars["psi_loss_p"])
    psi_loss_b = jvalue(vars["psi_loss_b"])
    psi_loss_c = jvalue(vars["psi_loss_c"])
    
    marge_dies = jvalue(vars["marge_dies"])
    
    L_PV = jvalue(vars["L_PV"]) # generated PV scenarios
    L_C  = jvalue(vars["L_C"]) # generated load scenarios
    L_C2 = reshape(mean(L_C,3),N_EMS,N_intra)

    # Defining iterators:
    ts  = 1:N_EMS
    ps  = 1:N_intra
    ds  = 1:N_dies
    ss  = 1:N_scen
    m1s = 1:N_firm
    m2s = (N_firm+1):N_EMS
    
    #----------------------------------------------------------------------------------------------------
    ### Variable definitions:

    @variables m begin

        # Battery variables:
        P_bat_min <= P_bat_set[t=ts,p=ps]  <= P_bat_max # Battery setpoint for time t,p (decision var.)
        0 <= P_bat_cha[t=ts,p=ps] <= P_bat_max
        0 <= P_bat_dis[t=ts,p=ps] <= P_bat_max
        X_bat[t=ts,p=ps], Bin                           # Operating mode indicator (decision var.)
                                                        # -> P_bat_set = 2*(X_bat-0.5)*P_bat_abs
        SOC_min <= SOC_bat[t=ts,p=ps] <= SOC_max        # Obtained SOC for time t,p (solution var.)

        # PV Plant:
        0 <= P_PV_set[t=ts,p=ps]  <= P_PV_inst          # PV setpoint variable (decision var.)
        0 <= P_PV[t=ts,p=ps,s=ss] <= L_PV[t,p,s]        # Actual PV generation (solution var.)

        # Diesel Generators:
        ON_dies[t=ts,d=ds], Bin                         # Dies. connectin setpoint (decision var.)
        0 <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max  # Actual dies. generation  (solution var.)
        # P_dies_min <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max # Actual dies. generation  (solution var.)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Variable Initialization (from previous optimization)
    
    if my_int>1

        vars = read_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(my_int-1)*".mat")

        P_bat_set_pre = jvalue(vars["P_bat_set"])[2:N_EMS,ps]
        SOC_bat_pre   = jvalue(vars["SOC_bat"])[2:N_EMS,ps]
        P_PV_set_pre  = jvalue(vars["P_PV_set"])[2:N_EMS,ps]
        P_PV_pre      = jvalue(vars["P_PV"])[2:N_EMS,ps,ss]
        ON_dies_pre   = jvalue(vars["ON_dies"])[2:N_EMS,ds]
        P_dies_pre    = jvalue(vars["P_dies"])[2:N_EMS,ps,ss,ds]
        P_bat_cha_pre = jvalue(vars["P_bat_cha"])[2:N_EMS,ps]
        P_bat_dis_pre = jvalue(vars["P_bat_dis"])[2:N_EMS,ps]
        X_bat_pre     = jvalue(vars["X_bat"])[2:N_EMS,ps]

        setvalue(P_bat_set[1:(N_EMS-1),ps],P_bat_set_pre)
        setvalue(SOC_bat[1:(N_EMS-1),ps],SOC_bat_pre)
        setvalue(P_PV_set[1:(N_EMS-1),ps],P_PV_set_pre)
        setvalue(P_PV[1:(N_EMS-1),ps,ss],P_PV_pre)
        setvalue(ON_dies[1:(N_EMS-1),ds],ON_dies_pre)
        setvalue(P_dies[1:(N_EMS-1),ps,ss,ds],P_dies_pre)
        setvalue(P_bat_cha[1:(N_EMS-1),ps],P_bat_cha_pre)
        setvalue(P_bat_dis[1:(N_EMS-1),ps],P_bat_dis_pre)
        setvalue(X_bat[1:(N_EMS-1),ps],X_bat_pre)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Constraints introduction:
    
    @constraints m begin

        # Constraints:
        #
        # A1 - Diesel power limitation
        # A2 - Diesel power limitation
        # B1 - Power balance equation
        # C1 - PV setpoint limitation
        # D1 - Bat. setpoint equation
        # D2 - Charging    mode const.
        # D3 - Discharging mode const.
        # E1 - FIRM    Diesel reserve margin (separated over scenarios)
        # E2 - RELAXED Diesel reserve margin (mean over scenarios)
        # F1 - 
        # F2 - SOC limitations
        # F3 - 
        # G1 - Minimum frequency limitation
        # X1 - Minimum voltage relaxed limitation
        # X2 - Minimum voltage firm    limitation

        A1[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] <= ON_dies[t,d]*(P_dies_max)
        A2[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] >= ON_dies[t,d]*(P_dies_min)

        # B1[t=ts,p=ps,s=ss], (P_PV[t,p,s] + sum(P_dies[t,p,s,d] for d=ds) + P_bat_set[t,p] - L_C[t,p,s]) == 0
        B1[t=ts,p=ps,s=ss], (P_PV[t,p,s]*(1-psi_loss_p) + sum((P_dies[t,p,s,d] - psi_loss_d*1e3*ON_dies[t,d]) for d=ds) + P_bat_set[t,p]*(1-psi_loss_b) - L_C[t,p,s]*(1+psi_loss_c) - psi_loss_i*1e3) == 0
                
        C1[t=ts,p=ps,s=ss],  P_PV[t,p,s] <= P_PV_set[t,p]

        D1[t=ts,p=ps], P_bat_set[t,p] == P_bat_cha[t,p] - P_bat_dis[t,p]
        D2[t=ts,p=ps], P_bat_cha[t,p] <= P_bat_max *    X_bat[t,p]
        D3[t=ts,p=ps], P_bat_dis[t,p] <= P_bat_max * (1-X_bat[t,p])

        # E1[t=ts,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E1[t=m1s,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E2[t=m2s,p=ps],  sum(P_dies[t,p,s,d] for d=ds,s=ss)/N_scen <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies

        F1[t=1,p=1],               SOC_bat[t,p] == SOC_init             - P_bat_set[t,p]*delta_t/Cap_bat
        F2[t=2:N_EMS,p=1],         SOC_bat[t,p] == SOC_bat[t-1,N_intra] - P_bat_set[t,p]*delta_t/Cap_bat
        F3[t=1:N_EMS,p=2:N_intra], SOC_bat[t,p] == SOC_bat[t,p-1]       - P_bat_set[t,p]*delta_t/Cap_bat

        G1[t=ts,p=ps], f_min <= theta_i + theta_d*sum(ON_dies[t,d] for d=ds) + theta_b*P_bat_set[t,p] + theta_p*P_PV_set[t,p]
        
        # X1[t=ts ,p=ps,s=ss], psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*P_PV[t,p,s] + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*L_C[t,p,s] >= U_min_def
        # X1[t=m1s,p=ps,s=ss], psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*P_PV[t,p,s]                      + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*L_C[t,p,s]                      >= U_min_def
        # X2[t=m2s,p=ps],      psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*sum(P_PV[t,p,s] for s=ss)/N_scen + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*sum(L_C[t,p,s] for s=ss)/N_scen >= U_min_def
        
    end

    #----------------------------------------------------------------------------------------------------
    ### Objective function definition:
    
    @objective(m, Max, sum((sum(P_PV[t,p,s] for s=ss)-N_scen*(1-eta_bat)*(P_bat_cha[t,p]+P_bat_dis[t,p])) for t=ts,p=ps));
    
    #----------------------------------------------------------------------------------------------------
    ### Solving Optimization:
    status = solve(m)

    #----------------------------------------------------------------------------------------------------
    ### Saving Data:
    
    # If Solved => Write Down the Results
    if string(status)!="Infeasible"
        write_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(interval)*".mat"; 
            status    = string(status),
            interval  = interval,
            P_bat_set = getvalue(P_bat_set[:,:]), 
            SOC_bat   = getvalue(SOC_bat[:,:]),
            P_PV_set  = getvalue(P_PV_set[:,:]),
            P_PV      = getvalue(P_PV[:,:,:]),
            ON_dies   = getvalue(ON_dies[:,:]),
            P_dies    = getvalue(P_dies[:,:,:,:]),
            P_bat_cha = getvalue(P_bat_cha[:,:]),
            P_bat_dis = getvalue(P_bat_dis[:,:]),
            X_bat     = getvalue(X_bat[:,:]),
            L_C       = L_C2)
    # If Infeasible -> Break!!!
    else 
        println("Interval "*dec(my_int)*" Infeasible -> Terminating Optimization!!!")
        break
    end
    
    # IJulia.clear_output(wait=false)
    println("Current progress: "*dec(my_int)*"/"*dec(N_EMS))
    
    my_int+=1
    
end

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [6e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-02, 1e+04]
Presolve removed 3667 rows and 4391 columns
Presolve time: 2.00s
Presolved: 192893 rows, 99001 columns, 682364 nonzeros
Variable types: 96409 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   25233    1.7485656e+07   1.268405e+06   0.000000e+00      5s
   50483    1.7464732e+07   7.994275e+05   0.000000e+00     10s
   74482    1.7433671e+07   6.144826e+05   0.000000e+00     15s
   94966    1.7382174e+07   4.356503e+05   0.000000e+00     20s
  115646    1.7240446e+07   3.390188e+05   0.000000e+00     25s
  135348    1.7014241e+07   1.530133e+05   0.000000e+00     30s
  156852  

# OPF Diverged  (n=11, pv=0 bat=21.4608 con=68.1234)
# OPF Diverged  (n=11, pv=0 bat=23.4372 con=75.2942)
Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [6e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 115.735245314

Presolve removed 3664 rows and 4388 columns
Presolve time: 1.93s
Presolved: 192896 rows, 99004 columns, 682376 nonzeros
Variable types: 96412 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7607871e+07   1.048947e+07   0.000000e+00      5s
   42805    1.7442942e+07   9.736923e+05   0.000000e+00     10s
   69225    1.7410635e+07   6.641524e+05   0.000000e

# OPF Diverged  (n=11, pv=-42.6092 bat=976.4839 con=12431.2378)
# OPF Diverged  (n=9, pv=-42.6092 bat=977.3648 con=12438.1557)
Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [6e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 609.135258214

Presolve removed 3667 rows and 4391 columns
Presolve time: 2.12s
Presolved: 192893 rows, 99001 columns, 682374 nonzeros
Variable types: 96409 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7488546e+07   1.048553e+07   0.000000e+00      5s
   47484    1.7326240e+07   8.724150e+05   0.000000e+00     10s
   68164    1.7302462e+07   6.4

# OPF Diverged  (n=9, pv=-85.1491 bat=2850.3417 con=24578.1358)
# OPF Diverged  (n=9, pv=-85.1491 bat=2850.3417 con=24584.6811)
Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [6e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 105.521680397

Presolve removed 3691 rows and 4415 columns
Presolve time: 1.95s
Presolved: 192869 rows, 98977 columns, 682316 nonzeros
Variable types: 96385 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   14502    1.7770580e+07   3.044161e+06   0.000000e+00      5s
   45992    1.7607609e+07   8.613288e+05   0.000000e+00     10s
   70219    1.7581130e+07   6.

# OPF Diverged  (n=9, pv=-126.8424 bat=4112.6529 con=36119.8278)
# OPF Diverged  (n=9, pv=-126.8424 bat=4112.6529 con=36126.1132)
Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [6e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 591.863339586

Presolve removed 3695 rows and 4419 columns
Presolve time: 1.82s
Presolved: 192865 rows, 98973 columns, 682314 nonzeros
Variable types: 96381 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7560773e+07   1.048227e+07   0.000000e+00      5s
   47345    1.7389221e+07   8.908049e+05   0.000000e+00     10s
   73857    1.7353966e+07   

...

# OPF Diverged  (n=10, pv=184376.5351 bat=4224.7325 con=531860.5644)
# OPF Diverged  (n=12, pv=184376.5351 bat=4224.7325 con=531869.0017)
Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [6e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [8e-01, 2e+04]
  RHS range        [2e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 388.988937199

Presolve removed 3691 rows and 4415 columns
Presolve time: 2.07s
Presolved: 192869 rows, 98977 columns, 682321 nonzeros
Variable types: 96385 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7683580e+07   1.048856e+07   0.000000e+00      6s
   38318    1.7517138e+07   9.702911e+05   0.000000e+00     10s
   63227    1.748205

# OPF Diverged  (n=12, pv=184355.9072 bat=4223.8076 con=539392.8767)
# OPF Diverged  (n=10, pv=184355.9072 bat=4223.8076 con=539401.3564)
Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [6e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 1168.851030165

Presolve removed 3703 rows and 4427 columns
Presolve time: 2.05s
Presolved: 192857 rows, 98965 columns, 682286 nonzeros
Variable types: 96373 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7736422e+07   1.048488e+07   0.000000e+00      6s
   38797    1.7570525e+07   9.617771e+05   0.000000e+00     10s
   63955    1.75387

# OPF Diverged  (n=10, pv=184313.9516 bat=4607.6394 con=554509.3252)
# OPF Diverged  (n=12, pv=184313.9516 bat=4607.6394 con=554517.5599)
Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [6e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 440.151909421

Presolve removed 3704 rows and 4428 columns
Presolve time: 2.11s
Presolved: 192856 rows, 98964 columns, 682285 nonzeros
Variable types: 96372 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7588888e+07   1.054762e+07   0.000000e+00      6s
   37911    1.7431348e+07   9.803082e+05   0.000000e+00     10s
   62026    1.740018

# OPF Diverged  (n=10, pv=184272.2891 bat=5222.8711 con=569270.6873)
# OPF Diverged  (n=11, pv=184272.2891 bat=5222.8711 con=569278.6711)


Excessive output truncated after 525719 bytes.